In [1]:
# 필요한 패키지 불러오기
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from gensim.models import FastText

# 데이터 로드
train_df = pd.read_csv('train_for_NLP.csv')
test_df = pd.read_csv('test_cleaned.csv')

# train 데이터에서 특성과 레이블 분리
X_train = train_df['combined_str']
y_train = train_df['target']

# test 데이터의 특성
X_test = test_df['combined_str']

# 하이퍼파라미터 그리드 설정
param_grid = {
    'C': [0.01, 0.1, 1, 3],  # 규제 강도
    'penalty': ['l1', 'l2'],  # 규제 유형
    'solver': ['liblinear']   # 최적화 알고리즘
}

# 문장을 단어 리스트로 분리
sentences = [text.split() for text in X_train]

# FastText 모델 학습 (Skipgram 사용)
fasttext_model = FastText(sentences, vector_size=100, window=5, sg=1, workers=4)

# FastText로 문서 벡터화
def vectorize_fasttext(text, model):
    words = text.split()
    # 각 문서 내 단어 벡터의 평균 계산
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    # 단어 벡터가 없는 경우에는 0으로 채운 벡터 반환
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# FastText로 변환된 데이터를 고정된 크기의 벡터로 변환
X_train_fasttext = np.array([vectorize_fasttext(text, fasttext_model) for text in X_train])
X_test_fasttext = np.array([vectorize_fasttext(text, fasttext_model) for text in X_test])

# 스케일링
scaler = StandardScaler()
X_train_fasttext_scaled = scaler.fit_transform(X_train_fasttext)
X_test_fasttext_scaled = scaler.transform(X_test_fasttext)

# GridSearchCV 적용
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='neg_log_loss', verbose=1)
grid_search.fit(X_train_fasttext_scaled, y_train)

# 최적 파라미터 및 성능 출력
print("FastText - Best Parameters:", grid_search.best_params_)
print("FastText - Best CV Log Loss:", -grid_search.best_score_)

# 테스트 데이터 예측
best_model = grid_search.best_estimator_
y_test_pred_prob_fasttext = best_model.predict_proba(X_test_fasttext_scaled)[:, 1]
y_test_pred_fasttext = (y_test_pred_prob_fasttext >= 0.5).astype(int)

# 제출 파일 생성
submission_df = pd.read_csv("sample_submission.csv")
submission_df['target'] = y_test_pred_fasttext
submission_df.to_csv("submit_FastText_Logistic.csv", index=False)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
FastText - Best Parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}
FastText - Best CV Log Loss: 0.5930340905021328
